In [1]:
import pandas as pd
from math import *

In [2]:
import glob, os
videos = glob.glob("open_field_model/open_field_zone-Me-2024-07-11/videos"+"/*.csv")
dataFiles = [pd.read_csv(f, skiprows=[0,0]) for f in videos]
#dataFiles = [pd.read_csv(f, skiprows=[0,0]) for f in glob.glob("open_field_model/open_field_zone-Me-2024-07-11/videos"+"/*.csv")]
video_name = []
for v in videos:
    name = v.split("Test")[1].split("DLC")[0]
    video_name.append(name)
    print(name)

 1
 20
 2
 3
 4
 5
 6
 7
 8
 9


Import of the differents datasets from the deeplabcut videos folder in a variable.

Create a video_name variable in order to make the link between the csv file and the video
(with the .csv file we don't know from what video it came from)

**dataFiles** contains the pandas read .csv
**video_name** contains the number of the video

dataFiles[ i ] correspond to video_name[ i ]

In [3]:
for data in dataFiles:
    print(data.shape)
    #print(data.head(3))

(31089, 10)
(31085, 10)
(31103, 10)
(31063, 10)
(31092, 10)
(31076, 10)
(31094, 10)
(31089, 10)
(31096, 10)
(31083, 10)


In [4]:
import mouse_size

la_ref_cm, la_ref_px, lo_ref_cm, lo_ref_px = mouse_size.main()
print("largeur de référence en centimètre : ",la_ref_cm)
print("largeur de référence en pixels : ",la_ref_px)
print("longueur de référence en centimètre : ",lo_ref_cm)
print("longueur de référence en pixels : ",lo_ref_px)

ratio = la_ref_px / la_ref_cm
print("Valeur en pixels pour 1 centimètre : ", ratio)

Dimensions in pixels: 89.3 px x 344.7 px
Dimensions in pixels: 89.3 px x 344.7 px
largeur de référence en centimètre :  3.366598722094302
largeur de référence en pixels :  89.26925562588724
longueur de référence en centimètre :  13.0
longueur de référence en pixels :  344.7100230628637
Valeur en pixels pour 1 centimètre :  26.51615562022028


Use of the mouse_size.py script that calculate the length of the mouse in pixel with its know width in centimeters.

The script is calibrated to work with the images in the images_for_openCV folder.

The images used for references need to be with only the mouse on it and it may need some change in the filters parameters used in the main function of the script in order to properly detect the mouse.

We used the info that a mouse length is approximately 13cm.

Source of the script (mine is a bit adapted) : https://github.com/Practical-CV/Measuring-Size-of-Objects-with-OpenCV

The script let us calculate the ratio of the image (use to calculate real centimeters from pixels length)

In [5]:
cmToPx = 1 #Number of pixels that represent 1cm
def acceleration(x1,y1,x2,y2,anc_acc):
    """
    Calculate the euclidian distance between two points and remove the former euclidian distance saved
    - x1 the previous position x
    - y1 the previous position y
    - x2 the actual position x
    - y2 the actual position y
    - anc_acc the previous acceleration
    
    Return the acceleration (acc) and the new previous acceleration (anc_acc)
    """
    x_diff = x2/ratio - x1/ratio
    y_diff = y2/ratio - y1/ratio
    acc = sqrt(pow(x_diff, 2) + pow(y_diff, 2))
    acc -= abs(anc_acc)
    #if acc < 0 and anc_acc >= 0 or acc >= 0 and anc_acc < 0:
    #    anc_acc = 0
    return (acc, anc_acc+acc)


In [26]:
minVitesses = []
maxVitesses = []
meanVitesses = []

minAccs = []
maxAccs = []
meanAccs = []

minDecs = []
maxDecs = []
meanDecs = []

vitesses = []
tabsAcc = []
for d in range(len(dataFiles)):
    data = dataFiles[d]
    tabAcc = []
    vitesse = []
    print("============================")
    teteX = data["Tete"]
    teteY = data["Tete.1"]

    timeOfVideo = 30

    previousTX = float(teteX[data.index.start+1])
    previousTY = float(teteY[data.index.start+1])

    previousAcc = 0

    anc_acc = 0

    minAcc = float("inf")
    maxAcc = float("-inf")
    meanAcc = 0

    minDec = float("inf")
    maxDec = float("-inf")
    meanDec = 0

    minVit = float("inf")
    maxVit = float("-inf")
    meanVit = 0

    distance = 0

    

    #number of frames of 1sec of the video
    fps = data.shape[0]//(timeOfVideo*60)
    #for every second of the video we calculate the acceleration
    for i in range(data.index.start+1, data.index.stop, 1):
        if i % fps == 0 and i > 1:
            minVit = min(vitesse[-fps:])
            maxVit = max(vitesse[-fps:])

            minVitesses.append(minVit)
            maxVitesses.append(maxVit)
            meanVitesses.append(meanVit/fps)

            minAccs.append(minAcc)
            maxAccs.append(maxAcc)
            meanAccs.append(meanAcc/fps)

            minDecs.append(minDec)
            maxDecs.append(maxDec)
            meanDecs.append(meanDec/fps)


            minVit = float("inf")
            maxVit = float("-inf")
            meanVit = 0

            minAcc = float("inf")
            maxAcc = float("-inf")
            meanAcc = 0

            minDec = float("inf")
            maxDec = float("-inf")
            meanDec = 0

        actualTX = float(teteX[i])
        actualTY = float(teteY[i])

        (acc, anc_acc) = acceleration(previousTX,previousTY,actualTX,actualTY,anc_acc)
        tabAcc.append(acc)

        distance += sqrt(pow(actualTX/ratio-previousTX/ratio, 2) + pow(actualTY/ratio-previousTY/ratio, 2))/100
        if acc >= 0:
            #acceleration
            minAcc = min(minAcc, acc)
            maxAcc = max(maxAcc, acc)
            meanAcc += acc
        else:
            #decceleration
            minDec = min(minDec, acc)
            maxDec = max(maxDec, acc)
            meanDec += acc
        
        meanVit += acc

        previousAcc = acc
        if len(vitesse) == 0:
            vitesse.append(acc)
        else:
            vitesse.append(vitesse[len(vitesse)-1] + acc)

        previousTX = actualTX
        previousTY = actualTY

    meanAcc = meanAcc/data.index.stop
    meanDec = meanDec/data.index.stop
    meanVit = meanVit/data.index.stop
    print(fps, " frames per second")
    print(fps*60, " frames per minutes")
    print("minimum acceleration = ",minAcc)
    print("maximum acceleration = ",maxAcc)
    print("mean acceleration = ",meanAcc)
    print("--------------------------------")
    print("minimum decceleration = ",minDec)
    print("maximum decceleration = ",maxDec)
    print("mean decceleration = ",meanDec)
    print("--------------------------------")
    print("minimum vitesse = ", min(vitesse))
    print("maximum vitesse = ", max(vitesse))
    print("mean vitesse = ", meanVit)
    print("--------------------------------")
    print("Distance crossed = ",distance)
    tabsAcc.append(tabAcc)
    vitesses.append(vitesse)


17  frames per second
1020  frames per minutes
minimum acceleration =  0.000360670100094513
maximum acceleration =  0.0225838583277057
mean acceleration =  2.452136072549522e-06
--------------------------------
minimum decceleration =  -0.055894854707208765
maximum decceleration =  -0.0009655171207570805
mean decceleration =  -4.427950065279607e-06
--------------------------------
minimum vitesse =  0.0
maximum vitesse =  2.460102269592179
mean vitesse =  -1.9758139927300845e-06
--------------------------------
Distance crossed =  68.12116312125293
17  frames per second
1020  frames per minutes
minimum acceleration =  0.03310468689524898
maximum acceleration =  0.09658558651401641
mean acceleration =  6.902232408400662e-06
--------------------------------
minimum decceleration =  -0.10461218198357243
maximum decceleration =  -0.0065658198956216746
mean decceleration =  -7.63576349529826e-06
--------------------------------
minimum vitesse =  0.0
maximum vitesse =  2.761202833566857
mea

For each dataFiles we calculate the acceleration (acc), speed (vitesse), and crossed distance (distance) of the mouse each second (time between each calculation can be change in the for i loop)

We set manually that the video is 30 minutes length (which is the case for my tests) and calculate the frame per seconds (fps) from this

Calculations are made from the head of the mouse only but can be change anytime from another point of the mouse (depending of the model used)

We save in an array all the acceleration, and speed in order to put them in graph and in an output in .csv

In [16]:
print(meanVitesses)

[0.018391217832364498, -0.012839850736107977, 0.01437718530047695, 0.013371250210652049, -0.027098407562304742, -0.0003468578780919499, 0.012362852821180144, -0.013149724654510194, 0.001693207827193316, -0.003779478295291023, 0.024116610002645592, -0.027084651848766655, 0.01872914193103745, -0.014587439497630087, 0.001623050030356421, 0.025213680437304195, -0.0008754798187800267, -0.014666400859202273, 0.00594619206663605, 0.004905381083178999, -0.002001204565456748, -0.018649428509681428, -0.0017712905270337498, 0.010622984121047365, 0.004845333322626892, -0.003752389959053748, -0.009219166689474357, 0.00101096706686766, -0.005907662447168257, 0.022176649246399404, 9.806666100833427e-05, 0.001733262092150699, -0.02204702449607034, 0.0034630659278233147, 0.021093862052791183, -0.022266440282775385, 0.007682230509481291, -0.003980301167498263, 0.002696003563127098, -0.009360713955420027, 0.00943102042521248, -0.004880306496171306, -0.005581931266625804, 0.013448075744880413, -0.00919583

In [28]:
import csv

repertoire = 'result_acceleration'
if not os.path.exists(repertoire):
    os.mkdir(repertoire)

for d in range(len(dataFiles)):
    with open(repertoire+'/Test_'+str(d)+'_out.csv', 'w+', newline='') as csvOut:
        writer = csv.writer(csvOut, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['tps','minVit','maxVit','meanVit','minAcc','maxAcc','meanAcc','minDec','maxDec','meanDec'])
        for i in range(len(minVitesses)):
            writer.writerow([i+1, minVitesses[i], maxVitesses[i], meanVitesses[i], minAccs[i], maxAccs[i], meanAccs[i], minDecs[i], maxDecs[i], meanDecs[i]])
        csvOut.close()


This cell write all the datas into .csv files in the result_acceleration folder (can be change at any time)

In [20]:
import plotly.graph_objects as plt
#import matplotlib.pyplot as plt

tabAcc = tabsAcc[0]
vitesse = vitesses[0]

choix = input("Choisissez un test :")
for i in range(len(video_name)):
    if choix == video_name[i]:
        tabAcc = tabsAcc[i]
        vitesse = vitesses[i]
        print("Choix réussi !")
        break

num_values_per_plot = 60
y_min, y_max = min(tabAcc), max(tabAcc)
for i in range(0, len(tabAcc), num_values_per_plot):
    fig = plt.Figure()
    chunk = tabAcc[i:i+num_values_per_plot]
    chunk_vit = vitesse[i:i+num_values_per_plot]

    
    x = list(range(i, i+len(chunk)))
    
    # Plot positive values
    pos_x = [x_val for x_val, y_val in zip(x, chunk) if y_val >= 0]
    pos_y = [y_val for y_val in chunk if y_val >= 0]
    fig.add_trace(plt.Scatter(x=pos_x, y=pos_y, mode='lines+markers', marker=dict(color='blue'), name='Accélération'))
    #plt.scatter(pos_x, pos_y, color='black', marker='x')  
    
    # Plot negative values
    neg_x = [x_val for x_val, y_val in zip(x, chunk) if y_val < 0]
    neg_y = [y_val for y_val in chunk if y_val < 0]
    fig.add_trace(plt.Scatter(x=neg_x, y=neg_y, mode='lines+markers', marker=dict(color='red'), name='Décélération'))
    #plt.scatter(neg_x, neg_y, color='black', marker='x') 

    # Plot vitesse
    fig.add_trace(plt.Scatter(x=x, y=chunk_vit, mode='lines+markers', marker=dict(color='green'), name='Vitesse'))
    
    fig.update_layout(
        title=f'Accélération de la souris de la frame {i} à la {i+num_values_per_plot-1}',
        xaxis_title='Frame',
        yaxis_title='Accélération',
        xaxis=dict(range=[i, i + num_values_per_plot]),
        yaxis=dict(range=[y_min, y_max]),
        legend=dict(x=0, y=1),
        showlegend=True,
        template="plotly_white"
    )
    fig.show()

We plot the acceleration, decceleration and speed of the desired dataFiles using plotly

We set with num_values_per_plot the number of values in each graph (to have a better visibility we set it to 60 to have a graph per minute of video)

In [7]:
print(tabsAcc[0])
print(vitesses[0])
print(len(tabsAcc[0]))
print(len(vitesses[0]))

[0.0, 0.3234861890373599, -0.24769748681441356, 0.30072316507681207, -0.09317840849438952, 0.04482566831880708, -0.16474606373457362, 0.10622324576567571, -0.0837147913242415, 0.09956454801375778, 0.0447365358976477, -0.0042233791295652745, -0.22167591118891417, 0.2252572407267086, 0.08032688871102489, -0.2664661650421193, 0.11273593899338347, -0.1885729894512454, -0.026460344114807424, 0.06109779630411957, -0.05426930412832923, 0.017597324921732452, 0.026026213286760677, 0.18345363445636342, 0.3519981380928033, -0.15200747245513319, -0.06255036881277382, -0.03801148916302216, -0.16424644816839634, 0.1425891622001177, 0.10915294510967294, 0.030274331467274596, -0.18300309561247258, 0.24996582649050914, 0.011963626335329547, -0.32345885964505017, 0.2006045629160117, 0.08078059037405372, -0.13359608161522551, -0.21935559661832466, 0.27303188667484524, -0.05676500738612483, -0.16290743541478372, 0.2537021708730217, -0.10792932287792867, -0.1946868554651753, 0.2450445120266104, 0.121237811